# JOINS 

This jupyter is to join the datasets with their 
one to many notes files.

In [1]:
import string
import bz2
from os import listdir
from os.path import isfile, join
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import datetime
import nltk.corpus  
import time
import math
import re
import spacy
from functools import reduce
from nltk.text import Text
from nltk.corpus import stopwords
from itertools import combinations
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
from textblob import TextBlob
from nltk.stem.wordnet import WordNetLemmatizer
# from keras.preprocessing import text, sequence
# from keras.utils import to_categorical
# from keras.layers import Bidirectional, GlobalMaxPool1D,GlobalMaxPool2D, GlobalMaxPool3D,Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation
# from keras.models import Model, Sequential
# from keras.callbacks import EarlyStopping, ModelCheckpoint
%matplotlib inline
plt.style.use('fivethirtyeight')


In [2]:
os.getcwd()

'C:\\Users\\t.cri.kcheung\\capstone_nlp\\src\\joins'

In [179]:
# get all files in subfolder
mypath = "Z:/FINAL OUTPUT v2/"
onlyfiles = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

# Modular

In [180]:
onlyfiles_sorted = sorted(onlyfiles, key=lambda x:(x!="PATIENT_COHORT", x))

In [181]:
onlyfiles_sorted

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/mer

In [13]:
onlyfiles_sorted_edit = list( onlyfiles_sorted[i] for i in [0,4,6,8,12,9] )
onlyfiles_sorted_edit

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_cohort_1_radiology.csv']

# check on thr length of total population - 1.2M rows

In [52]:
tempdf_check = pd.read_csv(onlyfiles_sorted[11],sep="|")

In [54]:
tempdf_check.shape

(1275765, 5)

### Prior to loading in lets check for j84 in part 2

In [16]:
onlyfiles_sorted[13]

'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv'

In [21]:
cohort2_icd_check = pd.read_csv(onlyfiles_sorted[13],sep="|")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [22]:
# get list of columns with dx
dx_cols = [col for col in cohort2_icd_check.columns if 'dx_' in col]
dx_cols[0]

'dx_1'

In [23]:
# all icd codes list
icd_codes_list = ['495','4951','4952','4953','4954','4955','4956','4957','4958','4959','496','515','517','5171','5172','5173','5178','516','5160','5161','5162','5163','51630','51631','51632','51633','51634','51635','51636','51637','5164','5165','5166','51661','51662','51663','51664','51669','5168','5169','J670','J671','J672','J673','J674','J675','J676','J677','J678','J679','J6710','J8410','J8489','J17','M3481','J99','J8401','J8402','J8403','J84111','J84112','J84113','J84114','J84115','J84116','J84117','J8481','J8482','J8483','J84841','J84842','J84843','J84844','J84845','J84848','J8409','J849']
pattern = '|'.join(icd_codes_list)


In [24]:
# extract all
icd_subset = []

for col in dx_cols:
#     print(col)
    tmpdf = cohort2_icd_check[cohort2_icd_check[col].str.contains(pattern,na=False)]
    icd_subset.append(tmpdf)
# print(icd_subset)
    

In [25]:
icd_subset_df = icd_subset[0]

In [26]:
len(np.unique(icd_subset_df.PATIENT_ID))

2243

# Now i have all the patients with the relevant icd codes. Around 2k unique of them.

We need to extract all of the relevant records into their own df. Let's tweak the merging into just those of which that are in the lsit of patients.

In [27]:
onlyfiles_sorted

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_cohort_1_radiology.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/08_cohort_2_radiology_header.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/mer

In [28]:
cohort2move = pd.read_csv(onlyfiles_sorted[3],sep="|")
cohort2move.head()

,PATIENT_ID,AGE_AT_LAST_ENCOUNTER,gender,race,ETHNICITY,DATE_OF_DEATH_D
0,49,79.0,M,White,Not Hispanic or Latino,NaN
1,99,86.0,F,White,Not Hispanic or Latino,NaN
2,104,84.0,M,White,Not Hispanic or Latino,NaN
3,114,66.0,F,White,Not Hispanic or Latino,NaN
4,118,92.0,F,Black/African-American,Not Hispanic or Latino,NaN


In [29]:
cohort2movedf = cohort2move[cohort2move.PATIENT_ID.isin(icd_subset_df.PATIENT_ID)]

In [30]:
cohort2movedf.to_csv(join(mypath,"01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv"),sep="|",index=False)

### Prior to loading in the second cohort let's randomly sample in 10k rows


In [31]:
cohort2move.head()

,PATIENT_ID,AGE_AT_LAST_ENCOUNTER,gender,race,ETHNICITY,DATE_OF_DEATH_D
0,49,79.0,M,White,Not Hispanic or Latino,NaN
1,99,86.0,F,White,Not Hispanic or Latino,NaN
2,104,84.0,M,White,Not Hispanic or Latino,NaN
3,114,66.0,F,White,Not Hispanic or Latino,NaN
4,118,92.0,F,Black/African-American,Not Hispanic or Latino,NaN


In [32]:
cohort2removeddf = cohort2move[~cohort2move.PATIENT_ID.isin(icd_subset_df.PATIENT_ID)]

In [33]:
cohort2sampdf = cohort2removeddf.sample(n=30000,random_state=10,replace=False)
cohort2sampdf.shape

(30000, 6)

In [34]:
cohort2sampdf.to_csv(join(mypath,"01_PATIENT_COHORT_2_NEW.csv"),sep="|",index=False)

In [36]:
del cohort2move, cohort2_icd_check,cohort2move

NameError: name 'cohort2_icd_check' is not defined

# Next step we need to create a join list with just the ids in moved from cohort2 set

Then we can merge them into a consolidated cohort1

In [111]:
onlyfiles2 = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [112]:
onlyfiles2

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2_sampled.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv',
 'Z:/FINAL OUTPUT v2/12

In [113]:
onlyfiles2edit = list( onlyfiles2[i] for i in [0, 8, 9, 15,16,18] )
onlyfiles2edit

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv']

In [102]:
# test
def test(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientids)]
            return(df)

In [103]:
patientlist = [114]

In [104]:
test(onlyfiles2edit,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv


,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death_d
0,114,66.0,F,White,Not Hispanic or Latino,NaN


### Edited fx

For mem purposes let's amend each read in to be just where it matches the patient id string

In [189]:
patientlist = np.unique(icd_subset_df['PATIENT_ID'])
patientlist[0:5]

array([ 114,  431,  672, 1408, 1549], dtype=int64)

In [106]:
frame = pd.DataFrame()
list_ = []

def readin_select(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientlist)]
            list_.append(df)

In [114]:
readin_select(onlyfiles2edit,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv
added to list_ Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (6,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv
added to list_ Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv


In [115]:
list_[0].iloc[1:5,:]

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death_d
1,431,88.0,F,Black/African-American,Not Hispanic or Latino,NaN
2,672,80.0,M,Asian/Mideast Indian,Not Hispanic or Latino,NaN
3,1408,84.0,F,Black/African-American,Not Hispanic or Latino,NaN
4,1549,80.0,F,Black/African-American,Not Hispanic or Latino,NaN


In [116]:
list_[5].iloc[1:5,:]

,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death_d
1,431,88.0,F,Black/African-American,Not Hispanic or Latino,NaN
2,672,80.0,M,Asian/Mideast Indian,Not Hispanic or Latino,NaN
3,1408,84.0,F,Black/African-American,Not Hispanic or Latino,NaN
4,1549,80.0,F,Black/African-American,Not Hispanic or Latino,NaN


In [117]:
len(list_[5])

2243

In [118]:
# spot check on patient ids 
list_[5][list_[5].patient_id==1408]
list_[0][list_[0].patient_id==1380083]


,patient_id,age_at_last_encounter,gender,race,ethnicity,date_of_death_d
879,1380083,65.0,F,Black/African-American,Not Hispanic or Latino,2014-10-13 00:00:00.000


# Correctly appending in function

Let's now see if we can use dask to improve the join functionality of the other datasets given memory issues.

In [119]:
df1 = list_[0]
df2 = list_[5]

In [120]:
print(df1.head())
print(len(df1))

   patient_id  age_at_last_encounter gender                    race  \
0         114                   66.0      F                   White   
1         431                   88.0      F  Black/African-American   
2         672                   80.0      M    Asian/Mideast Indian   
3        1408                   84.0      F  Black/African-American   
4        1549                   80.0      F  Black/African-American   

                ethnicity date_of_death_d  
0  Not Hispanic or Latino             NaN  
1  Not Hispanic or Latino             NaN  
2  Not Hispanic or Latino             NaN  
3  Not Hispanic or Latino             NaN  
4  Not Hispanic or Latino             NaN  
2243


In [121]:
print(df2.head())
print(len(df2))

   patient_id  age_at_last_encounter gender                    race  \
0         114                   66.0      F                   White   
1         431                   88.0      F  Black/African-American   
2         672                   80.0      M    Asian/Mideast Indian   
3        1408                   84.0      F  Black/African-American   
4        1549                   80.0      F  Black/African-American   

                ethnicity date_of_death_d  
0  Not Hispanic or Latino             NaN  
1  Not Hispanic or Latino             NaN  
2  Not Hispanic or Latino             NaN  
3  Not Hispanic or Latino             NaN  
4  Not Hispanic or Latino             NaN  
2243


Let's redo the function now with the read in and merge.

In [125]:
frame = pd.DataFrame()
list_ = []

def readin_select_full(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientlist)].dropna(axis='columns',how='all')
            list_.append(df)
#     suffix = ('_patient', '_encounter', '_encounter_provider', '_encounter_social','_dx_info','_radiology')
    suffix = ('_patient','_radiology')
    for i in range(0,len(list_)):
        print("iteration number",i)
        tmp1 = pd.DataFrame(list_[i].iloc[:,0])
        print("suffix : ",suffix[i])
        list_[i] = list_[i].iloc[:,1:].add_suffix(suffix[i])
        list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
#     frame = reduce(lambda x, y: pd.merge(x, y, on = 'patient_id',how="left"), list_)
    frame = reduce(lambda x, y: pd.merge(x, y, on = ['patient_id'],how="left"), list_)
    print(len(frame))
    return(frame)

In [123]:
onlyfiles3edit = list( onlyfiles2[i] for i in [0,18] )
onlyfiles3edit

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv']

In [126]:
readin_select_full(onlyfiles3edit,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv
added to list_ Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv
iteration number 0
suffix :  _patient
iteration number 1
suffix :  _radiology
59659


,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id_radiology,order_proc_id_deid_radiology,ordering_date_off_radiology,exam_type_radiology
0,114,66.0,F,White,Not Hispanic or Latino,NaN,4645531,602134,2013-08-22 00:00:00.000,XR Chest PA/Lateral
1,114,66.0,F,White,Not Hispanic or Latino,NaN,5066736,23950,2013-08-22 00:00:00.000,XR Chest PA/Lateral
2,114,66.0,F,White,Not Hispanic or Latino,NaN,5066736,232524,2013-08-22 00:00:00.000,XR Chest PA/Lateral
3,114,66.0,F,White,Not Hispanic or Latino,NaN,5097355,659564,2014-01-11 00:00:00.000,XR Chest PA/Lateral
4,114,66.0,F,White,Not Hispanic or Latino,NaN,6214618,307902,2014-06-22 00:00:00.000,CT Chest wo
5,114,66.0,F,White,Not Hispanic or Latino,NaN,6221749,852004,2014-12-06 00:00:00.000,XR Chest PA/Lateral
6,114,66.0,F,White,Not Hispanic or Latino,NaN,6221749,1237379,2014-12-06 00:00:00.000,XR Chest PA/Lateral
7,114,66.0,F,White,Not Hispanic or Latino,NaN,6265020,1179114,2014-01-11 00:00:00.000,XR Chest PA/Lateral
8,114,66.0,F,White,Not Hispanic or Latino,NaN,6265020,1145655,2014-01-11 00:00:00.000,XR Chest PA/Lateral
9,114,66.0,F,White,Not Hispanic or Latino,NaN,6747726,846507,2014-06-28 00:00:00.000,CT Chest wo


Looks good now. Full run. Comment out the suffix.

# Merge first grouping

In [127]:
onlyfiles1 = [join(mypath,f) for f in listdir(mypath) if isfile(join(mypath, f))]

In [128]:
onlyfiles1

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2_sampled.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv',
 'Z:/FINAL OUTPUT v2/12

In [129]:
# ORDER NOW MATTERS
onlyfiles1 = list( onlyfiles1[i] for i in [3,13,5,2] )
onlyfiles1

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv']

In [193]:
patientlist = np.unique(pd.read_csv(onlyfiles1[0],sep="|").PATIENT_ID)
patientlist[0:5]

array([ 32124,  74912,  81545,  93110, 118647], dtype=int64)

# Function def

Function to populate a list of dfs.

In [131]:
def get_list(listoffiles,like,patientids):
    for f in listoffiles:
        if f.find(like) == -1:
            print("skipped:",f)
        else:
            print("added to list_",f)
            df = pd.read_csv(f,index_col=None, header=0,sep="|")
            df.columns = [x.lower() for x in df.columns]
            df = df[df['patient_id'].isin(patientlist)]
            list_.append(df)
    suffix = ('_patient', '_encounter','_dx_info','radiology')
    for i in range(0,len(list_)):
        print("iteration number",i)
        print("suffix : ",suffix[i])
        if i == 0:
            tmp1 = pd.DataFrame(list_[i].iloc[:,0])
            list_[i] = list_[i].iloc[:,1:].add_suffix(suffix[i])
            list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
            print("dimensions : ", list_[i].shape)
        else:
            tmp1 = pd.DataFrame(list_[i].iloc[:,0:2])
            list_[i] = list_[i].iloc[:,2:].add_suffix(suffix[i])
            list_[i] = pd.concat((tmp1,(list_[i])),axis=1)
            print("dimensions : ", list_[i].shape)

In [132]:
def merge_lists(data):
    for i in range(0,len(list_)):
        print(i)
        if i == 1:
            print("merging first list slice: ", i) 
            print(data[i].iloc[0:1,:])
            df1 = data[0]
            df2 = data[i]
            holdingdf = pd.merge(df1, df2, how='left', on=['patient_id'])
        if i > 1:
            print("merging next list slice: ", i)
            print(data[i].iloc[0:1,:])
            df1 = holdingdf
            df2 = data[i]
            holdingdf = pd.merge(df1, df2, how='left', on=['patient_id','encounter_id'])
    return(holdingdf)

# Now run list and merge functions on pop one

In [140]:
list_[2].iloc[0:5]

,patient_id,encounter_id,dx_1,dx_desc_1,dx1_rank,dx_2,dx_desc_2,dx2_rank,dx_3,dx_desc_3,...,dx10_rank,dx_11,dx_desc_11,dx11_rank,dx_12,dx_desc_12,dx12_rank,dx_13,dx_desc_13,dx13_rank
0,32124,5039107,715.09,OSTEOARTHROSIS GENERALIZED INVOLVING MULTIPLE ...,1,275.49,OTHER DISORDERS OF CALCIUM METABOLISM,2.0,712.31,CHONDROCALCINOSIS CAUSE UNSPECIFIED INVOLVING ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74912,5237464,714.0,RHEUMATOID ARTHRITIS,1,715.09,OSTEOARTHROSIS GENERALIZED INVOLVING MULTIPLE ...,2.0,515,POSTINFLAMMATORY PULMONARY FIBROSIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,81545,4429153,516.34,RESPIRATORY BRONCHIOLITIS INTERSTITIAL LUNG DI...,1,495.9,UNSPECIFIED ALLERGIC ALVEOLITIS AND PNEUMONITIS,2.0,173.60,UNSPECIFIED MALIGNANT NEOPLASM OF SKIN OF UPPE...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,93110,5226111,714.81,RHEUMATOID LUNG,1,516.34,RESPIRATORY BRONCHIOLITIS INTERSTITIAL LUNG DI...,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,118647,6862790,786.05,SHORTNESS OF BREATH,1,311,DEPRESSIVE DISORDER NOT ELSEWHERE CLASSIFIED,2.0,710.1,SYSTEMIC SCLEROSIS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
onlyfiles1

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv']

In [168]:
list_ = []
get_list(onlyfiles1,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv
added to list_ Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv
added to list_ Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv
added to list_ Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv
iteration number 0
suffix :  _patient
dimensions :  (126, 6)
iteration number 1
suffix :  _encounter
dimensions :  (250, 19)
iteration number 2
suffix :  _dx_info
dimensions :  (250, 41)
iteration number 3
suffix :  radiology
dimensions :  (132, 5)


In [169]:
cohort1dta = merge_lists(list_)

0
1
merging first list slice:  1
   patient_id  encounter_id enc_eio_encounter visit_status_1_encounter  \
0       32124       5039107                 O                Completed   

  visit_status_2_encounter visit_status_3_encounter  visit_status_4_encounter  \
0                      NaN                      NaN                       NaN   

      adm_date_d_encounter    disc_date_d_encounter   start_date_d_encounter  \
0  2012-08-01 00:00:00.000  2012-08-01 00:00:00.000  2012-08-01 00:00:00.000   

      end_date_d_encounter admit_source_1_encounter admit_source_2_encounter  \
0  2012-08-01 00:00:00.000                      NaN                      NaN   

   admit_source_3_encounter disch_disp_1_encounter  disch_disp_2_encounter  \
0                       NaN                    NaN                     NaN   

   disch_disp_3_encounter  zip_d_encounter enc_fin_class_encounter  
0                     NaN           100058                Medicare  
2
merging next list slice:  2
   patie

In [170]:
cohort1dta.head()

,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,visit_status_2_encounter,...,dx11_rank_dx_info,dx_12_dx_info,dx_desc_12_dx_info,dx12_rank_dx_info,dx_13_dx_info,dx_desc_13_dx_info,dx13_rank_dx_info,order_proc_id_deidradiology,ordering_date_offradiology,exam_type radiology
0,32124,90.0,F,White,Not Hispanic or Latino,NaN,5039107,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,74912,63.0,F,Black/African-American,Not Hispanic or Latino,NaN,5237464,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,81545,65.0,M,White,Not Hispanic or Latino,2015-06-21 00:00:00.000,4429153,O,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,93110,47.0,F,Black/African-American,Not Hispanic or Latino,2013-04-07 00:00:00.000,5226111,O,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,118647,52.0,F,Black/African-American,Not Hispanic or Latino,NaN,6862790,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,475790.0,2016-05-11 00:00:00.000,XR Chest PA/Lateral\t\t


In [171]:
cohort1dta.to_csv(join(mypath,"merged_cohort1.csv"),sep="|",index=False)

# Quick note

Differences in dx columns. Need to account for that in the union of the two.

# Below we merge the moved from cohort 2 grouping

In [196]:
# redefine patients
patientlist = np.unique(icd_subset_df['PATIENT_ID'])
print(patientlist[0:5])
print(len(patientlist))

[ 114  431  672 1408 1549]
2243


In [183]:
onlyfiles_sorted

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/mer

In [185]:
# ORDER NOW MATTERS
onlyfiles3editshort = list( onlyfiles_sorted[i] for i in [1,5,13,10] )
onlyfiles3editshort

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv']

In [197]:
list_ = []

get_list(onlyfiles3editshort,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv
added to list_ Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (6,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv
iteration number 0
suffix :  _patient
dimensions :  (2243, 6)
iteration number 1
suffix :  _encounter
dimensions :  (18868, 19)
iteration number 2
suffix :  _dx_info
dimensions :  (18868, 260)
iteration number 3
suffix :  radiology
dimensions :  (59659, 5)


In [198]:
holdingdf = pd.DataFrame()
cohort1_dta_part2 = merge_lists(list_)

0
1
merging first list slice:  1
    patient_id  encounter_id enc_eio_encounter visit_status_1_encounter  \
42         114       4645531                 O                Completed   

   visit_status_2_encounter visit_status_3_encounter visit_status_4_encounter  \
42                 Canceled                      NaN                      NaN   

       adm_date_d_encounter    disc_date_d_encounter   start_date_d_encounter  \
42  2013-08-22 00:00:00.000  2013-08-22 00:00:00.000  2013-08-22 00:00:00.000   

       end_date_d_encounter admit_source_1_encounter admit_source_2_encounter  \
42  2013-08-22 00:00:00.000                      NaN                      NaN   

   admit_source_3_encounter disch_disp_1_encounter disch_disp_2_encounter  \
42                      NaN                    NaN                    NaN   

   disch_disp_3_encounter  zip_d_encounter enc_fin_class_encounter  
42                    NaN           111723        Medicaid - Major  
2
merging next list slice:  2
   p

In [199]:
cohort1_dta_part2[cohort1_dta_part2['patient_id']==114]

,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,visit_status_2_encounter,...,dx84_rank_dx_info,dx_85_dx_info,dx_desc_85_dx_info,dx85_rank_dx_info,dx_86_dx_info,dx_desc_86_dx_info,dx86_rank_dx_info,order_proc_id_deidradiology,ordering_date_offradiology,exam_typeradiology
0,114,66.0,F,White,Not Hispanic or Latino,NaN,4645531,O,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,602134.0,2013-08-22 00:00:00.000,XR Chest PA/Lateral
1,114,66.0,F,White,Not Hispanic or Latino,NaN,5066736,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23950.0,2013-08-22 00:00:00.000,XR Chest PA/Lateral
2,114,66.0,F,White,Not Hispanic or Latino,NaN,5066736,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232524.0,2013-08-22 00:00:00.000,XR Chest PA/Lateral
3,114,66.0,F,White,Not Hispanic or Latino,NaN,5097355,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,659564.0,2014-01-11 00:00:00.000,XR Chest PA/Lateral
4,114,66.0,F,White,Not Hispanic or Latino,NaN,6214618,O,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307902.0,2014-06-22 00:00:00.000,CT Chest wo
5,114,66.0,F,White,Not Hispanic or Latino,NaN,6221749,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852004.0,2014-12-06 00:00:00.000,XR Chest PA/Lateral
6,114,66.0,F,White,Not Hispanic or Latino,NaN,6221749,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1237379.0,2014-12-06 00:00:00.000,XR Chest PA/Lateral
7,114,66.0,F,White,Not Hispanic or Latino,NaN,6265020,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1179114.0,2014-01-11 00:00:00.000,XR Chest PA/Lateral
8,114,66.0,F,White,Not Hispanic or Latino,NaN,6265020,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1145655.0,2014-01-11 00:00:00.000,XR Chest PA/Lateral
9,114,66.0,F,White,Not Hispanic or Latino,NaN,6747726,O,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,846507.0,2014-06-28 00:00:00.000,CT Chest wo


In [201]:
print(cohort1_dta_part2.shape)
cohort1_dta_part2.head()

(59662, 285)


,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,visit_status_2_encounter,...,dx84_rank_dx_info,dx_85_dx_info,dx_desc_85_dx_info,dx85_rank_dx_info,dx_86_dx_info,dx_desc_86_dx_info,dx86_rank_dx_info,order_proc_id_deidradiology,ordering_date_offradiology,exam_typeradiology
0,114,66.0,F,White,Not Hispanic or Latino,NaN,4645531,O,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,602134.0,2013-08-22 00:00:00.000,XR Chest PA/Lateral
1,114,66.0,F,White,Not Hispanic or Latino,NaN,5066736,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23950.0,2013-08-22 00:00:00.000,XR Chest PA/Lateral
2,114,66.0,F,White,Not Hispanic or Latino,NaN,5066736,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,232524.0,2013-08-22 00:00:00.000,XR Chest PA/Lateral
3,114,66.0,F,White,Not Hispanic or Latino,NaN,5097355,O,Completed,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,659564.0,2014-01-11 00:00:00.000,XR Chest PA/Lateral
4,114,66.0,F,White,Not Hispanic or Latino,NaN,6214618,O,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,307902.0,2014-06-22 00:00:00.000,CT Chest wo


We finally have a merged dataset that should be clean of dupes.

In [202]:
cohort1_dta_part2.to_csv(join(mypath,"merged_movedfromcohort1.csv"),sep="|")

# Merge of the first and second cohort1 groupings.

Note that there's two different column sets


In [30]:
pd.set_option('display.max_columns', None)

In [203]:
# if from before. read in csvs

cohort1dta = pd.read_csv(join(mypath,"merged_cohort1.csv"),sep="|")
cohort1_dta_part2=pd.read_csv(join(mypath,"merged_movedfromcohort1.csv"),sep="|")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,22,109,110,112,113,115,116,118,119,121,122,124,125,127,128,130,131,133,134,136,137,139,140,142,143,145,146,148,149,151,152,154,155,157,158,160,161,163,164,166,167,169,170,172,173,175,176,178,179,181,182,184,185,187,188,190,191,193,194,196,197,199,200,202,203,205,206,208,209,211,212,214,215,217,218,220,221,223,224) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [204]:
cohort1dta[cohort1dta.patient_id==360469].columns

Index(['patient_id', 'age_at_last_encounter_patient', 'gender_patient',
       'race_patient', 'ethnicity_patient', 'date_of_death_d_patient',
       'encounter_id', 'enc_eio_encounter', 'visit_status_1_encounter',
       'visit_status_2_encounter', 'visit_status_3_encounter',
       'visit_status_4_encounter', 'adm_date_d_encounter',
       'disc_date_d_encounter', 'start_date_d_encounter',
       'end_date_d_encounter', 'admit_source_1_encounter',
       'admit_source_2_encounter', 'admit_source_3_encounter',
       'disch_disp_1_encounter', 'disch_disp_2_encounter',
       'disch_disp_3_encounter', 'zip_d_encounter', 'enc_fin_class_encounter',
       'dx_1_dx_info', 'dx_desc_1_dx_info', 'dx1_rank_dx_info', 'dx_2_dx_info',
       'dx_desc_2_dx_info', 'dx2_rank_dx_info', 'dx_3_dx_info',
       'dx_desc_3_dx_info', 'dx3_rank_dx_info', 'dx_4_dx_info',
       'dx_desc_4_dx_info', 'dx4_rank_dx_info', 'dx_5_dx_info',
       'dx_desc_5_dx_info', 'dx5_rank_dx_info', 'dx_6_dx_info',
       'd

In [205]:
cohort1dta.rename(columns={'exam_type\t\tradiology':'exam_typeradiology'}, inplace=True)

In [206]:
cohort1_dta_part2[cohort1_dta_part2.patient_id==2959602].columns

Index(['Unnamed: 0', 'patient_id', 'age_at_last_encounter_patient',
       'gender_patient', 'race_patient', 'ethnicity_patient',
       'date_of_death_d_patient', 'encounter_id', 'enc_eio_encounter',
       'visit_status_1_encounter',
       ...
       'dx84_rank_dx_info', 'dx_85_dx_info', 'dx_desc_85_dx_info',
       'dx85_rank_dx_info', 'dx_86_dx_info', 'dx_desc_86_dx_info',
       'dx86_rank_dx_info', 'order_proc_id_deidradiology',
       'ordering_date_offradiology', 'exam_typeradiology'],
      dtype='object', length=286)

In [207]:
cohort1_dta_part2[cohort1_dta_part2.patient_id==2959602][['patient_id','exam_typeradiology']]

,patient_id,exam_typeradiology
59648,2959602,CT Chest wo
59649,2959602,CT Chest wo
59650,2959602,IOE COM XR CHEST
59651,2959602,IOE COM CT ANG CHEST
59652,2959602,IOE COM XR CHEST
59653,2959602,IOE COM CT CHEST
59654,2959602,"XR Port Chest, 1 View"
59655,2959602,"XR Port Chest, 1 View"
59656,2959602,IOE COM CT ANG CHEST
59657,2959602,IOE COM CT CHEST


In [208]:
cohort1total = pd.merge(cohort1dta,cohort1_dta_part2,how="outer")

In [209]:
cohort1total.to_csv(join(mypath,"merged_cohort1_withcohort2.csv"),sep="|")

# Now merge for the cohort2 sample group

In [211]:
onlyfiles_sorted

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_1_MOVEDFROMCOHORT2.csv',
 'Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/02_PATIENT_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/03_ENCOUNTER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/05_ENCOUNTER_PROVIDER_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/06_ENCOUNTER_PROVIDER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/07_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/07_ENCOUNTER_SOCIAL_HX_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv',
 'Z:/FINAL OUTPUT v2/08_ENCOUNTER_SOCIAL_HX_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/09_DX_INFO_COHORT_1.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/11_COHORT_1_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/12_COHORT_2_RADIOLOGY.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv',
 'Z:/FINAL OUTPUT v2/merged_cohort2.csv',
 'Z:/FINAL OUTPUT v2/mer

In [215]:
# ORDER NOW MATTERS
onlyfilessampled = list( onlyfiles_sorted[i] for i in [2,5,13,10] )
onlyfilessampled

['Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv',
 'Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv',
 'Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv']

In [216]:
patientlist = np.unique(cohort2sampdf.PATIENT_ID)
print(patientlist[0:5])
print(len(patientlist))

[ 99 195 219 256 560]
30000


In [217]:
list_ = []
get_list(onlyfilessampled,"COHORT",patientlist)

added to list_ Z:/FINAL OUTPUT v2/01_PATIENT_COHORT_2_NEW.csv
added to list_ Z:/FINAL OUTPUT v2/04_ENCOUNTER_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (6,13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/10_DX_INFO_COHORT_2.csv


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2856: DtypeWarning: Columns (98,110,113,116,117,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228,230,231,233,234,236,237,239,240,242,243,245,246,248,249,251,252,254,255,257,258) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


added to list_ Z:/FINAL OUTPUT v2/08_COHORT_2_RADIOLOGY_HEADER.csv
iteration number 0
suffix :  _patient
dimensions :  (30000, 6)
iteration number 1
suffix :  _encounter
dimensions :  (95743, 19)
iteration number 2
suffix :  _dx_info
dimensions :  (95743, 260)
iteration number 3
suffix :  radiology
dimensions :  (307165, 5)


In [218]:
holdingdf = pd.DataFrame()
cohort2dta = merge_lists(list_)

0
1
merging first list slice:  1
    patient_id  encounter_id enc_eio_encounter visit_status_1_encounter  \
21          99       5169683                 O                Completed   

   visit_status_2_encounter visit_status_3_encounter visit_status_4_encounter  \
21                      NaN                      NaN                      NaN   

       adm_date_d_encounter    disc_date_d_encounter   start_date_d_encounter  \
21  2013-04-06 00:00:00.000  2013-04-06 00:00:00.000  2013-04-06 00:00:00.000   

       end_date_d_encounter     admit_source_1_encounter  \
21  2013-04-06 00:00:00.000  Clinic or Physicians Office   

   admit_source_2_encounter admit_source_3_encounter disch_disp_1_encounter  \
21                      NaN                      NaN                    NaN   

   disch_disp_2_encounter disch_disp_3_encounter  zip_d_encounter  \
21                    NaN                    NaN           149109   

     enc_fin_class_encounter  
21  Blue Cross / Blue Shield  
2
merging

In [219]:
cohort2dta.head()

,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,visit_status_2_encounter,...,dx84_rank_dx_info,dx_85_dx_info,dx_desc_85_dx_info,dx85_rank_dx_info,dx_86_dx_info,dx_desc_86_dx_info,dx86_rank_dx_info,order_proc_id_deidradiology,ordering_date_offradiology,exam_typeradiology
0,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,911368.0,2013-10-11 00:00:00.000,"XR Port Chest, 1 View"
1,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1144261.0,2013-10-11 00:00:00.000,CT Head wo
2,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,907325.0,2013-10-11 00:00:00.000,CT Neck Soft Tissue wo
3,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,643685.0,2013-10-11 00:00:00.000,CT Head wo
4,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,Canceled,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,666840.0,2013-10-11 00:00:00.000,CT Head wo


In [220]:
cohort2dta.to_csv(join(mypath,"merged_cohort2_sampled.csv"),sep="|")

# Joining to rad notes  

string dict is "encounter_id" and "order_proc_id_deid"

In [325]:
# read csvs if skipping above
cohort1total=pd.read_csv("Z:/FINAL OUTPUT v2/merged_cohort1_withcohort2.csv",sep="|")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (19,21,22,116,119,120,122,123,125,126,128,129,131,132,134,135,137,138,140,141,143,144,146,147,149,150,152,153,155,156,158,159,161,162,164,165,167,168,170,171,173,174,176,177,179,180,182,183,185,186,188,189,191,192,194,195,197,198,200,201,203,204,206,207,209,210,212,213,215,216,218,219,221,222,224,225,227,228) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
cohort2dta=pd.read_csv("Z:/FINAL OUTPUT v2/merged_cohort2_sampled.csv",sep="|")

In [222]:
# extract encounter_id from notes
mypathnotes = "Z:/notes v2/"
onlyfilesnotes = [join(mypathnotes,f) for f in listdir(mypathnotes) if isfile(join(mypathnotes, f))]
onlyfilesnotes_sorted = sorted(onlyfilesnotes, key=lambda x:(x!="cohort", x))
print(onlyfilesnotes_sorted)

['Z:/notes v2/cohort1_deid_df.csv', 'Z:/notes v2/cohort1_deid_df_editedforchecks.xlsx', 'Z:/notes v2/cohort2_deid_df.csv', 'Z:/notes v2/cohort_1_v2.zip', 'Z:/notes v2/cohort_2_v2.zip', 'Z:/notes v2/parsing_draft_v2.py']


In [396]:
# cohort 1
notes1 = pd.read_csv(onlyfilesnotes_sorted[0],index_col=None, header=0,sep=",")
notes1.columns = [x.lower() for x in notes1.columns]

In [397]:
# cohort 2
notes2 = pd.read_csv(onlyfilesnotes_sorted[2],index_col=None, header=0,sep=",",error_bad_lines=False)
notes2.columns = [x.lower() for x in notes2.columns]

In [398]:
# split 1
notes1[['encounter_id','order_proc_id_deidradiology']] = notes1['patient'].str.split('_',expand=True)

# split2
notes2[['encounter_id','order_proc_id_deidradiology']] = notes2['patient'].str.split('_',expand=True)

In [370]:
notes1.head(n=2)

,unnamed: 0,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext,encounter_id,order_proc_id_deidradiology
0,0,"XR CHEST PA/LATERAL,",2014-10-18**]> 9:58 AM,**NAME<BBB> **AGE<in 60s> years old. Status po...,XR CHEST PA/LATERAL,**DATE<[**2014-09-20**]>.,No pneumothorax. Irregular opacity along the ...,No pneumothorax. Nonspecific air space opacit...,**DATE<[**2014-10-18**]> 12:13 PM [**Doctor Fi...,5035743_1018855,"XR CHEST PA/LATERAL, **DATE<[**2014-10-18**]> ...",5035743,1018855
1,1,"XR CHEST PA/LATERAL,",2015-03-15**]> 12:55 PM,Cough and weight loss,Two view chest,**DATE<[**2013-07-01**]>,Stable predominately peripheral and lower zone...,Chronic interstitial lung disease an appearanc...,**DATE<[**2015-03-15**]> 1:15 PM **NAME<GGG HH...,4981923_729470,"XR CHEST PA/LATERAL, **DATE<[**2015-03-15**]> ...",4981923,729470


# KC NOTE

issue with nulls and integer values ( not implemented in pandas....).

Trying convert back to numeric to allow nulls.

In [399]:
# cohort1total.order_proc_id_deidradiology = cohort1total.order_proc_id_deidradiology.astype(float)
cohort1total.order_proc_id_deidradiology[0:5]

0         NaN
1         NaN
2         NaN
3         NaN
4    475790.0
Name: order_proc_id_deidradiology, dtype: float64

In [400]:
notes1.order_proc_id_deidradiology[0:5]
# pd.to_numeric(notes1.order_proc_id_deidradiology[0:5])

0    1018855
1     729470
2     596007
3     625450
4     479362
Name: order_proc_id_deidradiology, dtype: object

In [401]:
# ensure int type
results = [float(i) for i in notes1.encounter_id]
notes1.encounter_id = results

resultsorder = [float(i) for i in notes1.order_proc_id_deidradiology]
notes1.order_proc_id_deidradiology = resultsorder

In [402]:
notes1.order_proc_id_deidradiology[0:5]
# pd.to_numeric(notes1.order_proc_id_deidradiology[0:5])

0    1018855.0
1     729470.0
2     596007.0
3     625450.0
4     479362.0
Name: order_proc_id_deidradiology, dtype: float64

In [403]:
# ensure int type
results = [float(i) for i in notes2.encounter_id]
notes2.encounter_id = results

resultsorder = [float(i) for i in notes2.order_proc_id_deidradiology]
notes2.order_proc_id_deidradiology = resultsorder

In [404]:
notes1[notes1.encounter_id==5035743]

,unnamed: 0,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext,encounter_id,order_proc_id_deidradiology
0,0,"XR CHEST PA/LATERAL,",2014-10-18**]> 9:58 AM,**NAME<BBB> **AGE<in 60s> years old. Status po...,XR CHEST PA/LATERAL,**DATE<[**2014-09-20**]>.,No pneumothorax. Irregular opacity along the ...,No pneumothorax. Nonspecific air space opacit...,**DATE<[**2014-10-18**]> 12:13 PM [**Doctor Fi...,5035743_1018855,"XR CHEST PA/LATERAL, **DATE<[**2014-10-18**]> ...",5035743.0,1018855.0


In [377]:
cohort1total[cohort1total.encounter_id==5035743]

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,dx83_rank_dx_info,dx_84_dx_info,dx_desc_84_dx_info,dx84_rank_dx_info,dx_85_dx_info,dx_desc_85_dx_info,dx85_rank_dx_info,dx_86_dx_info,dx_desc_86_dx_info,dx86_rank_dx_info
22,22,360469,65.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,23,360469,65.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11607,11607,360469,66.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11608,11608,360469,66.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [405]:
cohort1total[cohort1total.order_proc_id_deidradiology==1018855]

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,dx83_rank_dx_info,dx_84_dx_info,dx_desc_84_dx_info,dx84_rank_dx_info,dx_85_dx_info,dx_desc_85_dx_info,dx85_rank_dx_info,dx_86_dx_info,dx_desc_86_dx_info,dx86_rank_dx_info
23,23,360469,65.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11608,11608,360469,66.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# convert to floats for types

some issues were present with types. Cohort 1 matched just fine with the fixed notes in excel.

### Now join notes for cohort1

In [408]:
cohort1_dta_notes = pd.merge(cohort1total, notes1, on =['encounter_id','order_proc_id_deidradiology'],how="left")

In [409]:
cohort1_dta_notes.head(n=5)

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext
0,0,32124,90.0,F,White,Not Hispanic or Latino,NaN,5039107,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,74912,63.0,F,Black/African-American,Not Hispanic or Latino,NaN,5237464,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,81545,65.0,M,White,Not Hispanic or Latino,2015-06-21 00:00:00.000,4429153,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,93110,47.0,F,Black/African-American,Not Hispanic or Latino,2013-04-07 00:00:00.000,5226111,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,118647,52.0,F,Black/African-American,Not Hispanic or Latino,NaN,6862790,O,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [410]:
cohort1_dta_notes[cohort1_dta_notes.impressions.notnull()]

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext
16,16,174454,83.0,M,Black/African-American,Not Hispanic or Latino,2014-05-27 00:00:00.000,4672267,O,Completed,...,CT CHEST WO,2015-04-17**]> 2:00 PM,83-year-old male with shortness of breath,"Non-enhanced chest CT, Multiplanar reconstruc...",**DATE<[**2014-06-10**]>,LUNGS AND PLEURA: Patchy subpleural reticulati...,Patchy areas of fibrosis and honeycombing most...,"**NAME<KKK LLL>, MD I personally reviewed the...",4672267_738614,CT CHEST WO **DATE<[**2015-04-17**]> 2:00 PM\...
20,20,239207,32.0,F,Black/African-American,Not Hispanic or Latino,2013-01-12 00:00:00.000,5197877,E,Completed,...,"XR PORT CHEST 1V,",2015-06-05**]> 6:07 AM,Reason: r/o pulm edema History: missed hd,XR PORT CHEST 1V,**DATE<[**2015-04-24**]>,Cardiomegaly with diffuse nonspecific airspace...,Extensive bilateral chronic abnormalities with...,**DATE<[**2015-06-05**]> 8:58 AM **NAME<AAA B...,5197877_1069870,"XR PORT CHEST 1V, **DATE<[**2015-06-05**]> 6:0..."
23,23,360469,65.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,"XR CHEST PA/LATERAL,",2014-10-18**]> 9:58 AM,**NAME<BBB> **AGE<in 60s> years old. Status po...,XR CHEST PA/LATERAL,**DATE<[**2014-09-20**]>.,No pneumothorax. Irregular opacity along the ...,No pneumothorax. Nonspecific air space opacit...,**DATE<[**2014-10-18**]> 12:13 PM [**Doctor Fi...,5035743_1018855,"XR CHEST PA/LATERAL, **DATE<[**2014-10-18**]> ..."
25,25,360469,65.0,M,Black/African-American,Not Hispanic or Latino,NaN,5317316,O,Completed,...,"XR CHEST PA/LATERAL,",2016-09-03**]> 10:33 AM,**NAME<BBB> **AGE<in 60s> years old. Reason: s...,XR CHEST PA/LATERAL,**DATE<[**2014-12-13**]> and previous.,Unremarkable mediastinal and cardiac silhouett...,No significant abnormality. No significant cha...,**DATE<[**2016-09-03**]> 10:35 AM **NAME<CCC D...,5317316_696052,"XR CHEST PA/LATERAL, **DATE<[**2016-09-03**]> ..."
31,31,563961,24.0,M,Black/African-American,Not Hispanic or Latino,NaN,5254604,O,Completed,...,CT CHEST WO,2015-07-20**]> 10:05 AM,24-year-old male with findings suggesting chro...,"Non-enhanced chest CT, Multiplanar reconstruc...",None available.,LUNGS AND PLEURA: Volume loss of the right low...,1. Volume loss and mosaic perfusion as detaile...,"**NAME<CCC DDD>, MD I personally reviewed the...",5254604_145704,CT CHEST WO **DATE<[**2015-07-20**]> 10:05 AM...
35,35,563961,24.0,M,Black/African-American,Not Hispanic or Latino,NaN,5500610,O,Completed,...,"XR CHEST PA/LATERAL,",2015-07-19**]> 1:48 PM,NaN,XR CHEST PA/LATERAL,**DATE<[**2015-06-25**]>.,Low lung volumes. Chronic appearing right bas...,Persistent chronic appearing right basal inter...,**DATE<[**2015-07-19**]> 5:00 PM [**Doctor Las...,5500610_201903,"XR CHEST PA/LATERAL, **DATE<[**2015-07-19**]> ..."
37,37,643150,34.0,M,Black/African-American,Not Hispanic or Latino,NaN,4829821,O,Completed,...,CT CHEST W,2015-07-03**]> 10:35 AM,34-year-old male with interstitial process on ...,Contrast enhanced chest CT. 120 mL of Omnipaq...,**DATE<[**2015-03-20**]>,LUNGS AND PLEURA: Background diffuse groundgla...,1. No significant change in diffuse groundglas...,"**NAME<CCC DDD>, MD I personally reviewed the...",4829821_153569,CT CHEST W **DATE<[**2015-07-03**]> 10:35 AM\...
40,40,663052,81.0,M,Black/African-American,Not Hispanic or Latino,NaN,6277220,O,Completed,...,CT CHEST WO,2017-05-14**]> 10:25 AM,Reason: Evaluate for progression of ILD Histo...,"Non-enhanced chest CT, Multiplanar reconstruc...",**DATE<[**2012-09-11**]>,LUNGS AND PLEURA: Upper zone predominant chron...,Slight interval progression off upper zone pre...,**DATE<[**2017-05-14**]> 1:21 PM **NAME<EEE F...,6277220_584198,CT CHEST WO **DATE<[**2017-05-14**]> 10:25 AM...
48,48,812480,50.0,F,More than one Race,Hispanic or Latino,201

### Merge works with floats!

Now to join the second.

Second however ends up joining on multiple. Ideally i'd want a coalesce.

In [411]:
cohort1_dta_notes_tmp = pd.merge(cohort1_dta_notes, notes2, on =['encounter_id','order_proc_id_deidradiology'],how="left")

In [412]:
cohort1_dta_notes_tmp.columns.values

array(['Unnamed: 0', 'patient_id', 'age_at_last_encounter_patient',
       'gender_patient', 'race_patient', 'ethnicity_patient',
       'date_of_death_d_patient', 'encounter_id', 'enc_eio_encounter',
       'visit_status_1_encounter', 'visit_status_2_encounter',
       'visit_status_3_encounter', 'visit_status_4_encounter',
       'adm_date_d_encounter', 'disc_date_d_encounter',
       'start_date_d_encounter', 'end_date_d_encounter',
       'admit_source_1_encounter', 'admit_source_2_encounter',
       'admit_source_3_encounter', 'disch_disp_1_encounter',
       'disch_disp_2_encounter', 'disch_disp_3_encounter',
       'zip_d_encounter', 'enc_fin_class_encounter', 'dx_1_dx_info',
       'dx_desc_1_dx_info', 'dx1_rank_dx_info', 'dx_2_dx_info',
       'dx_desc_2_dx_info', 'dx2_rank_dx_info', 'dx_3_dx_info',
       'dx_desc_3_dx_info', 'dx3_rank_dx_info', 'dx_4_dx_info',
       'dx_desc_4_dx_info', 'dx4_rank_dx_info', 'dx_5_dx_info',
       'dx_desc_5_dx_info', 'dx5_rank_dx_info', 'dx_

In [419]:
# coalesce
cohort1_dta_notes.scandate = cohort1_dta_notes_tmp.scandate_x.combine_first(cohort1_dta_notes_tmp.scandate_y)
cohort1_dta_notes.findings = cohort1_dta_notes_tmp.findings_x.combine_first(cohort1_dta_notes_tmp.findings_y)
cohort1_dta_notes.impressions = cohort1_dta_notes_tmp.impressions_x.combine_first(cohort1_dta_notes_tmp.impressions_y)

In [420]:
cohort1_dta_notes[cohort1_dta_notes.impressions.notnull()]

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext
16,16,174454,83.0,M,Black/African-American,Not Hispanic or Latino,2014-05-27 00:00:00.000,4672267,O,Completed,...,CT CHEST WO,2015-04-17**]> 2:00 PM,83-year-old male with shortness of breath,"Non-enhanced chest CT, Multiplanar reconstruc...",**DATE<[**2014-06-10**]>,LUNGS AND PLEURA: Patchy subpleural reticulati...,Patchy areas of fibrosis and honeycombing most...,"**NAME<KKK LLL>, MD I personally reviewed the...",4672267_738614,CT CHEST WO **DATE<[**2015-04-17**]> 2:00 PM\...
20,20,239207,32.0,F,Black/African-American,Not Hispanic or Latino,2013-01-12 00:00:00.000,5197877,E,Completed,...,"XR PORT CHEST 1V,",2015-06-05**]> 6:07 AM,Reason: r/o pulm edema History: missed hd,XR PORT CHEST 1V,**DATE<[**2015-04-24**]>,Cardiomegaly with diffuse nonspecific airspace...,Extensive bilateral chronic abnormalities with...,**DATE<[**2015-06-05**]> 8:58 AM **NAME<AAA B...,5197877_1069870,"XR PORT CHEST 1V, **DATE<[**2015-06-05**]> 6:0..."
23,23,360469,65.0,M,Black/African-American,Not Hispanic or Latino,NaN,5035743,O,Completed,...,"XR CHEST PA/LATERAL,",2014-10-18**]> 9:58 AM,**NAME<BBB> **AGE<in 60s> years old. Status po...,XR CHEST PA/LATERAL,**DATE<[**2014-09-20**]>.,No pneumothorax. Irregular opacity along the ...,No pneumothorax. Nonspecific air space opacit...,**DATE<[**2014-10-18**]> 12:13 PM [**Doctor Fi...,5035743_1018855,"XR CHEST PA/LATERAL, **DATE<[**2014-10-18**]> ..."
25,25,360469,65.0,M,Black/African-American,Not Hispanic or Latino,NaN,5317316,O,Completed,...,"XR CHEST PA/LATERAL,",2016-09-03**]> 10:33 AM,**NAME<BBB> **AGE<in 60s> years old. Reason: s...,XR CHEST PA/LATERAL,**DATE<[**2014-12-13**]> and previous.,Unremarkable mediastinal and cardiac silhouett...,No significant abnormality. No significant cha...,**DATE<[**2016-09-03**]> 10:35 AM **NAME<CCC D...,5317316_696052,"XR CHEST PA/LATERAL, **DATE<[**2016-09-03**]> ..."
31,31,563961,24.0,M,Black/African-American,Not Hispanic or Latino,NaN,5254604,O,Completed,...,CT CHEST WO,2015-07-20**]> 10:05 AM,24-year-old male with findings suggesting chro...,"Non-enhanced chest CT, Multiplanar reconstruc...",None available.,LUNGS AND PLEURA: Volume loss of the right low...,1. Volume loss and mosaic perfusion as detaile...,"**NAME<CCC DDD>, MD I personally reviewed the...",5254604_145704,CT CHEST WO **DATE<[**2015-07-20**]> 10:05 AM...
35,35,563961,24.0,M,Black/African-American,Not Hispanic or Latino,NaN,5500610,O,Completed,...,"XR CHEST PA/LATERAL,",2015-07-19**]> 1:48 PM,NaN,XR CHEST PA/LATERAL,**DATE<[**2015-06-25**]>.,Low lung volumes. Chronic appearing right bas...,Persistent chronic appearing right basal inter...,**DATE<[**2015-07-19**]> 5:00 PM [**Doctor Las...,5500610_201903,"XR CHEST PA/LATERAL, **DATE<[**2015-07-19**]> ..."
37,37,643150,34.0,M,Black/African-American,Not Hispanic or Latino,NaN,4829821,O,Completed,...,CT CHEST W,2015-07-03**]> 10:35 AM,34-year-old male with interstitial process on ...,Contrast enhanced chest CT. 120 mL of Omnipaq...,**DATE<[**2015-03-20**]>,LUNGS AND PLEURA: Background diffuse groundgla...,1. No significant change in diffuse groundglas...,"**NAME<CCC DDD>, MD I personally reviewed the...",4829821_153569,CT CHEST W **DATE<[**2015-07-03**]> 10:35 AM\...
40,40,663052,81.0,M,Black/African-American,Not Hispanic or Latino,NaN,6277220,O,Completed,...,CT CHEST WO,2017-05-14**]> 10:25 AM,Reason: Evaluate for progression of ILD Histo...,"Non-enhanced chest CT, Multiplanar reconstruc...",**DATE<[**2012-09-11**]>,LUNGS AND PLEURA: Upper zone predominant chron...,Slight interval progression off upper zone pre...,**DATE<[**2017-05-14**]> 1:21 PM **NAME<EEE F...,6277220_584198,CT CHEST WO **DATE<[**2017-05-14**]> 10:25 AM...
48,48,812480,50.0,F,More than one Race,Hispanic or Latino,201

In [423]:
# write
cohort1_dta_notes.to_csv("Z:/final_data/merged_notes_cohort_1_final.csv",sep="|")

# Cohort 2

In [424]:
cohort2dta.order_proc_id_deidradiology[0:5]

0     911368.0
1    1144261.0
2     907325.0
3     643685.0
4     666840.0
Name: order_proc_id_deidradiology, dtype: float64

In [426]:
cohort2dta[cohort2dta.order_proc_id_deidradiology==911368]

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,dx84_rank_dx_info,dx_85_dx_info,dx_desc_85_dx_info,dx85_rank_dx_info,dx_86_dx_info,dx_desc_86_dx_info,dx86_rank_dx_info,order_proc_id_deidradiology,ordering_date_offradiology,exam_typeradiology
0,0,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,911368.0,2013-10-11 00:00:00.000,"XR Port Chest, 1 View"


### Now join notes for cohort2

In [428]:
cohort2_dta_notes = pd.merge(cohort2dta, notes2, on =['encounter_id','order_proc_id_deidradiology'],how="left")

In [429]:
cohort2_dta_notes.head(n=5)

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext
0,0,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,"XR PORT CHEST 1V,",2015-12-26**]> 4:15 AM,**NAME<ZZZ> **AGE<in 70s> years old. Reason: [...,XR PORT CHEST 1V,**DATE<[**2013-08-14**]>,Low lung volumes. No specific evidence of infe...,No specific evidence of infection or failure.,**DATE<[**2015-12-26**]> 11:51 AM **NAME<AAA B...,7872100_911368,"XR PORT CHEST 1V, **DATE<[**2015-12-26**]> 4:1..."
1,1,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,"CT HEAD WO,",2016-01-19**]> 9:05 PM,**AGE<in 70s> year old female with abnormality...,CT of the head without contrast.,**DATE<[**2014-09-20**]>,Periventricular hypoattenuation is reidentifie...,1. Stable edema and mass effect involving the ...,**DATE<[**2016-01-20**]> 10:41 AM **NAME<BBB C...,7872100_666840,"CT HEAD WO, **DATE<[**2016-01-19**]> 9:05 PM ..."


In [430]:
cohort2_dta_notes[cohort2_dta_notes.impressions.notnull()]

,Unnamed: 0,patient_id,age_at_last_encounter_patient,gender_patient,race_patient,ethnicity_patient,date_of_death_d_patient,encounter_id,enc_eio_encounter,visit_status_1_encounter,...,scantype,scandate,clininfo,technique,comparison,findings,impressions,elecsig,patient,rawtext
0,0,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,"XR PORT CHEST 1V,",2015-12-26**]> 4:15 AM,**NAME<ZZZ> **AGE<in 70s> years old. Reason: [...,XR PORT CHEST 1V,**DATE<[**2013-08-14**]>,Low lung volumes. No specific evidence of infe...,No specific evidence of infection or failure.,**DATE<[**2015-12-26**]> 11:51 AM **NAME<AAA B...,7872100_911368,"XR PORT CHEST 1V, **DATE<[**2015-12-26**]> 4:1..."
4,4,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,"CT HEAD WO,",2016-01-19**]> 9:05 PM,**AGE<in 70s> year old female with abnormality...,CT of the head without contrast.,**DATE<[**2014-09-20**]>,Periventricular hypoattenuation is reidentifie...,1. Stable edema and mass effect involving the ...,**DATE<[**2016-01-20**]> 10:41 AM **NAME<BBB C...,7872100_666840,"CT HEAD WO, **DATE<[**2016-01-19**]> 9:05 PM ..."
5,5,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,"CT HEAD WO, CT NECK SOFT TISSUE WO,",2015-12-28**]> 7:16 AM,"**NAME<EEE> **AGE<in 70s> years old, history o...",CT of the head and soft tissue neck without co...,**DATE<[**2014-08-29**]> HEAD CT \r\n\r\nFIND...,Hazy periventricular hypoattenuation is identi...,1. New edema and mass effect are detected in ...,**DATE<[**2015-12-28**]> 11:48 AM **NAME<FFF G...,7872100_200096,"CT HEAD WO, CT NECK SOFT TISSUE WO, **DATE<[**..."
6,6,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,"CT CHEST ABDOMEN PELVIS WO,",2015-12-24**]> 7:39 AM,**AGE<in 70s> year old female. NHL. Evaluate...,Helical CT chest chest abdomen pelvis. No ora...,**DATE<[**2014-08-25**]> and prior.,CHEST: \r\n\r\nLUNGS AND PLEURA: Few scattere...,No evidence of recurrent disease. Gas and fat...,**DATE<[**2015-12-24**]> 8:18 AM **NAME<ZZZ AA...,7872100_491973,"CT CHEST ABDOMEN PELVIS WO, **DATE<[**2015-12-..."
8,8,1843306,76.0,F,Black/African-American,Not Hispanic or Latino,2015-02-22 00:00:00.000,7872100,I,Completed,...,"CT HEAD WO, CT NECK SOFT TISSUE WO,",2015-12-30**]> 7:16 AM,"**NAME<DDD> **AGE<in 70s> years old, history o...",CT of the head and soft tissue neck without co...,**DATE<[**2014-08-31**]> HEAD CT \r\n\r\nFIND...,Hazy periventricular hypoattenuation is identi...,1. New edema and mass effect are detected in ...,**DATE<[**2015-12-30**]> 11:48 AM **NAME<EEE F...,7872100_323897,"CT HEAD WO, CT NECK SOFT TISSUE WO, **DATE<[**..."
10,10,1386150,79.0,M,Black/African-American,Not Hispanic or Latino,2014-03-20 00:00:00.000,5333739,"E, I",Completed,...,"XR CHEST PA/LATERAL,",2016-09-21**]> 10:53 PM,**NAME<BBB> **AGE<in 70s> years old. Reason: r...,XR CHEST PA/LATERAL,**DATE<[**2014-08-28**]>,No evidence of pneumonia. Stable cardiomegaly...,No evidence of pneumonia.,**DATE<[**2016-09-22**]> 10:40 AM **NAME<CCC D...,5333739_412567,"XR CHEST PA/LATERAL, **DATE<[**2016-09-21**]> ..."
12,12,1386150,79.0,M,Black/African-American,Not Hispanic or Latino,2014-03-20 00:00:00.000,5651131,I,Completed,...,"XR PORT CHEST 1V,",2016-07-27**]> 12:15 PM,**NAME<III> **AGE<in 70s> years old. Reason: e...,XR PORT CHEST 1V,**DATE<[**2016-06-21**]>,No focal air space opacity. Unremarkable card...,No acute cardiopulmonary abnormality. No evid...,"**NAME<JJJ KKK>, MD I personally reviewed the...",5651131_719490,"XR PORT CHEST 1V, **DATE<[**2016-07-27**]> 12:..."
15,15,1386150,79.0,M,Black/African-American,Not Hispanic or Latino,2014-03-20 00:00:00.000,5651131,I,Completed,...,CT UPPER ABD AND PELVIS WO,2016-04-23**]> 2:01 PM,Male; 79 years old. Reason: r/o worsening perf...,Nonenhanced CT images of the abdomen and pelvi...,CT ches

In [431]:
# write
cohort2_dta_notes.to_csv("Z:/final_data/merged_notes_cohort_2_final.csv",sep="|")